# PETsARD

## import PETsARD

In [1]:
import os
from pprint import pprint
import sys

sys.path.append(os.path.dirname(os.getcwd()))

import PETsARD

# Module-by-Module

## Loader

In [2]:
loader = PETsARD.Loader(
    filepath='benchmark://adult',
    na_values={k: '?' for k in [
        'workclass',
        'occupation',
        'native-country'
    ]}
)
print(loader.data.head(1))


Loader - Benchmarker: file benchmark\adult.csv already exist and match SHA-256.
                      PETsARD will ignore download and use local data directly.
   age workclass  fnlwgt education  educational-num marital-status  \
0   25   Private  226802      11th                7  Never-married   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  


## metadata

In [3]:
pprint(loader.metadata.metadata)

{'col': {'age': {'dtype': dtype('int8'),
                 'infer_dtype': 'numerical',
                 'na_percentage': 0.0},
         'capital-gain': {'dtype': dtype('int32'),
                          'infer_dtype': 'numerical',
                          'na_percentage': 0.0},
         'capital-loss': {'dtype': dtype('int16'),
                          'infer_dtype': 'numerical',
                          'na_percentage': 0.0},
         'education': {'dtype': CategoricalDtype(categories=['10th', '11th', '12th', '1st-4th', '5th-6th', '7th-8th',
                  '9th', 'Assoc-acdm', 'Assoc-voc', 'Bachelors', 'Doctorate',
                  'HS-grad', 'Masters', 'Preschool', 'Prof-school',
                  'Some-college'],
, ordered=False),
                       'infer_dtype': 'categorical',
                       'na_percentage': 0.0},
         'educational-num': {'dtype': dtype('int8'),
                             'infer_dtype': 'numerical',
                             'na_percent

## Splitter

In [4]:
splitter = PETsARD.Splitter(
    data=loader.data,
    num_samples=30,
    train_split_ratio=0.1
)
print(splitter.data[1]['train'].shape[0])
print(splitter.data[1]['validation'].shape[0])
print(splitter.data[1]['train'].head(1))
print(splitter.data[1]['validation'].head(1))


4884
43958
   age workclass  fnlwgt education  educational-num marital-status  \
0   25   Private  226802      11th                7  Never-married   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  
   age workclass  fnlwgt education  educational-num      marital-status  \
0   38   Private   89814   HS-grad                9  Married-civ-spouse   

        occupation relationship   race gender  capital-gain  capital-loss  \
0  Farming-fishing      Husband  White   Male             0             0   

   hours-per-week native-country income  
0              50  United-States  <=50K  


## Processor: transform()

In [5]:
processor = PETsARD.processor.Processor(
    metadata=loader.metadata,
)
processor.fit(
    data=splitter.data[1]['train'],
)
preproc_data = processor.transform(
    data=splitter.data[1]['train']
)
print(preproc_data.head(1))

No self-defined config passed.  Generate a config automatically.
        age  workclass    fnlwgt  education  educational-num  marital-status  \
0 -0.999057   0.307087  0.390396   0.824784        -1.181942        0.520919   

   occupation  relationship      race    gender  capital-gain  capital-loss  \
0    0.779348      0.796506  0.911432  0.405937     -0.146857     -0.212508   

   hours-per-week  native-country    income  
0       -0.032114        0.844928  0.610213  


## Synthesizer

### Normal: Cont. as 0~1

In [9]:
sdv_methods = [
    # 'sdv-singletable-copulagan',
    # 'sdv-singletable-ctgan',
    # 'sdv-singletable-gaussiancopula',
    # 'sdv-singletable-tvae'
]

smartnoise_methods = [
    'smartnoise-mwem',
]
# 可能由於版本限制，無法執行 aim
 # 'smartnoise-aim', 
# GAN系未支援
 # 'smartnoise-dpctgan',
 # 'smartnoise-patectgan',
 # 'smartnoise-dpgan',
 # 'smartnoise-pategan',

for synthesizing_method in sdv_methods + smartnoise_methods:
    print(f"Synthesizing method: {synthesizing_method}")
    synthesizer = PETsARD.Synthesizer(
        data=preproc_data,
        synthesizing_method=synthesizing_method,
        epsilon=10.0,
    )
    synthesizer.fit_sample()
    print(synthesizer.data_syn.head(1))

Synthesizing method: smartnoise-mwem
Synthesizer (SmartNoise): Fitting mwem.


d:\Dropbox\89_other_application\GitHub\PETsARD\.venv\lib\site-packages\snsynth\mwem.py:570: UserWarning: Data should be preprocessed to have 0 based indices.
  warnings.warn("Data should be preprocessed to have 0 based indices.")


Synthesizer (SmartNoise): Fitting  mwem spent 5.6909 sec.
Synthesizer (SmartNoise): Sampling mwem # 2796 rows (same as raw) in 0.4303 sec.
   age  workclass  fnlwgt  education  educational-num  marital-status  \
0    0          0      -1          0                0               0   

   occupation  relationship  race  gender  capital-gain  capital-loss  \
0           0             0     0       0             0             0   

   hours-per-week  native-country  income  
0               0               0       0  


### Categorical
for mst, pacsynth in smartnoise

`ValueError: The transformer appears to have some continuous columns. Please provide only categorical or ordinal.`

In [23]:
# processor_cate = PETsARD.processor.Processor(
#     metadata=loader.metadata,
# )

# metadata_col = loader.metadata.metadata['col']
# colnames_discrete = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country', 'income']
# processor_cate.update_config(
#     {'encoder': {col: 'encoder_label' if col in colnames_discrete else None for col in metadata_col},
#      'scaler': {col: None for col in metadata_col},
#      }
# )

# processor_cate.fit(
#     data=splitter.data[1]['train'],
#     sequence=None
# )
# preproc_data_cate = processor_cate.transform(
#     data=splitter.data[1]['train']
# )
# print(preproc_data_cate.head(1))

No self-defined config passed.  Generate a config automatically.
   age  workclass  fnlwgt  education  educational-num  marital-status  \
0   25          3  226802          1                7               4   

   occupation  relationship  race  gender  capital-gain  capital-loss  \
0           6             3     2       1             0             0   

   hours-per-week  native-country  income  
0              40              36       0  


In [30]:
# smartnoise_methods_cate = [
#     'smartnoise-mst',
#     'smartnoise-pacsynth',
# ]

# for col in preproc_data_cate.columns:
#     preproc_data_cate[col] = preproc_data_cate[col].astype('category')

# for synthesizing_method in smartnoise_methods_cate:
#     print(f"Synthesizing method: {synthesizing_method}")
#     synthesizer_cate = PETsARD.Synthesizer(
#         data=preproc_data_cate,
#         synthesizing_method=synthesizing_method,
#         epsilon=1.0,
#     )
#     synthesizer_cate.fit_sample()
#     print(synthesizer_cate.data_syn.head(1))

Synthesizing method: smartnoise-mst
Synthesizer (SmartNoise): Fitting mst.


ValueError: The transformer appears to have some continuous columns. Please provide only categorical or ordinal.

## Processor: inverse_transform()

In [7]:
postproc_data = processor.inverse_transform(
    data=synthesizer.data_syn
)
print(postproc_data.head(1))

ValueError: Bin edges must be unique: array([0.        , 0.91376797, 0.93230577, 0.94042908, 0.94480317,
       0.94896896, 0.95292647, 0.95625911, 0.95959175, 0.96292439,
       0.96625703, 0.96938138, 0.97229744, 0.97479692, 0.9772964 ,
       0.97958759, 0.98187878, 0.98396167, 0.98583628, 0.98771089,
       0.98916892, 0.99062695, 0.99187669, 0.99291814, 0.9937513 ,
       0.99458446, 0.99520933, 0.9958342 , 0.99645907, 0.99708394,
       0.99770881, 0.99812539, 0.99854197, 0.99875026, 0.99895855,
       0.99916684, 0.99937513, 0.99958342, 0.99979171, 1.        ,
       1.        , 1.        ]).
You can drop duplicate edges by setting the 'duplicates' kwarg

## Evaluator

### Anonymeter

In [11]:
eval = PETsARD.Evaluator(
    evaluating_method='anonymeter-singlingout-univariate',
    data={
        'ori': splitter.data[1]['train'],
        'syn': postproc_data,
        'control': splitter.data[1]['validation']
    },
    anonymeter_n_attacks=2 # 2000
)
eval.eval()
eval.Evaluator.evaluation

Evaluator (Anonymeter - SinglingOut - Univariate): Now is SinglingOut - Univariate Evaluator
Evaluator (Anonymeter - SinglingOut - Univariate): Evaluator time: 0.0734 sec.
Evaluator (Anonymeter): Evaluating  SinglingOut - Univariate.


{'Risk': 0.0,
 'Risk_CI_btm': 0.0,
 'Risk_CI_top': 0.6928102649963914,
 'Attack_Rate': 0.32880988624667346,
 'Attack_Rate_err': 0.32880988624667346,
 'Baseline_Rate': 0.32880988624667346,
 'Baseline_Rate_err': 0.32880988624667346,
 'Control_Rate': 0.32880988624667346,
 'Control_Rate_err': 0.32880988624667346}

In [12]:
eval = PETsARD.Evaluator(
    evaluating_method='anonymeter-linkability',
    data={
        'ori': splitter.data[1]['train'],
        'syn': postproc_data,
        'control': splitter.data[1]['validation']
    },
    anonymeter_n_attacks=2, # 2000,
    anonymeter_n_neighbors=10,
    anonymeter_n_jobs=-1,
    anonymeter_aux_cols=[
        ['age', 'fnlwgt', 'race', 'gender', 'native-country'],
        ['workclass', 'education', 'capital-gain', 'capital-loss', 'hours-per-week']
    ]
)
eval.eval()
eval.Evaluator.evaluation


Evaluator (Anonymeter - Linkability): Now is Linkability Evaluator
Evaluator (Anonymeter - Linkability): aux_cols are [age, fnlwgt, race, gender, native-country]
                                      and [workclass, education, capital-gain, capital-loss, hours-per-week].
Evaluator (Anonymeter - Linkability): Evaluator time: 0.0 sec.
Evaluator (Anonymeter): Evaluating  Linkability.
Evaluator (Anonymeter): Evaluating Linkability spent 8.6534 sec.


{'Risk': 0.0,
 'Risk_CI_btm': 0.0,
 'Risk_CI_top': 0.6928102649963914,
 'Attack_Rate': 0.32880988624667346,
 'Attack_Rate_err': 0.32880988624667346,
 'Baseline_Rate': 0.32880988624667346,
 'Baseline_Rate_err': 0.32880988624667346,
 'Control_Rate': 0.32880988624667346,
 'Control_Rate_err': 0.32880988624667346}

In [13]:
eval = PETsARD.Evaluator(
    evaluating_method='anonymeter-inference',
    data={
        'ori': splitter.data[1]['train'],
        'syn': postproc_data,
        'control': splitter.data[1]['validation']
    },
    anonymeter_n_attacks=2, #2000,
    anonymeter_n_jobs=-1,
    anonymeter_secret='age'
)
eval.eval()
eval.Evaluator.evaluation


Evaluator (Anonymeter - Inference): Now is Inference Evaluator
Evaluator (Anonymeter - Inference): Evaluator time: 0.0 sec.
Evaluator (Anonymeter): Evaluating  Inference.
Evaluator (Anonymeter): Evaluating Inference spent 0.6498 sec.


{'Risk': 0.0,
 'Risk_CI_btm': 0.0,
 'Risk_CI_top': 0.013449535585480343,
 'Attack_Rate': 0.08915647695108263,
 'Attack_Rate_err': 0.024683334547474907,
 'Baseline_Rate': 0.09114122827015952,
 'Baseline_Rate_err': 0.0249352437952306,
 'Control_Rate': 0.11495824409908227,
 'Control_Rate_err': 0.027695443904980018}

### SDMetrics

In [12]:
eval = PETsARD.Evaluator(
    evaluating_method='sdmetrics-single_table-qualityreport',
    data={
        'ori': splitter.data[1]['train'],
        'syn': postproc_data
    }
)
eval.eval()
eval.Evaluator.evaluation

Evaluator (SDMetrics): Evaluating QualityReport.
Generating report ...
(2/2) Evaluating Column Pair Trends: : 100%|██████████| 105/105 [00:05<00:00, 18.18it/s]

Overall Score: 74.06%

Properties:
- Column Shapes: 91.81%
- Column Pair Trends: 56.31%
Evaluator (SDMetrics): Evaluating QualityReport spent 6.0073 sec.


{'score': 0.7405721651768964,
 'properties': {'Column Shapes': {'Score': 0.918070881955508},
  'Column Pair Trends': {'Score': 0.5630734483982847}},
 'details': {'Column Shapes':              Column        Metric     Score
  0               age  KSComplement  0.948650
  1         workclass  TVComplement  0.996087
  2            fnlwgt  KSComplement  0.950445
  3         education  TVComplement  0.539399
  4   educational-num  KSComplement  0.881909
  5    marital-status  TVComplement  0.971009
  6        occupation  TVComplement  0.981831
  7      relationship  TVComplement  0.962217
  8              race  TVComplement  0.997025
  9            gender  TVComplement  0.998437
  10     capital-gain  KSComplement  0.918025
  11     capital-loss  KSComplement  0.952909
  12   hours-per-week  KSComplement  0.692668
  13   native-country  TVComplement  0.991754
  14           income  TVComplement  0.988698,
  'Column Pair Trends':            Column 1         Column 2                 Metric   

In [13]:
eval = PETsARD.Evaluator(
    evaluating_method='sdmetrics-single_table-diagnosticreport',
    data={
        'ori': splitter.data[1]['train'],
        'syn': postproc_data
    }
)
eval.eval()
eval.Evaluator.evaluation

Evaluator (SDMetrics): Evaluating DiagnosticReport.
Generating report ...
(2/2) Evaluating Data Structure: : 100%|██████████| 1/1 [00:00<?, ?it/s]

Overall Score: 100.0%

Properties:
- Data Validity: 100.0%
- Data Structure: 100.0%
Evaluator (SDMetrics): Evaluating DiagnosticReport spent 0.0853 sec.


{'score': 1.0,
 'properties': {'Data Validity': {'Score': 1.0},
  'Data Structure': {'Score': 1.0}},
 'details': {'Data Validity':              Column             Metric  Score
  0               age  BoundaryAdherence    1.0
  1         workclass  CategoryAdherence    1.0
  2            fnlwgt  BoundaryAdherence    1.0
  3         education  CategoryAdherence    1.0
  4   educational-num  BoundaryAdherence    1.0
  5    marital-status  CategoryAdherence    1.0
  6        occupation  CategoryAdherence    1.0
  7      relationship  CategoryAdherence    1.0
  8              race  CategoryAdherence    1.0
  9            gender  CategoryAdherence    1.0
  10     capital-gain  BoundaryAdherence    1.0
  11     capital-loss  BoundaryAdherence    1.0
  12   hours-per-week  BoundaryAdherence    1.0
  13   native-country  CategoryAdherence    1.0
  14           income  CategoryAdherence    1.0,
  'Data Structure':            Metric  Score
  0  TableStructure    1.0}}

# Executor

## run()

In [15]:
import os
from pprint import pprint
import sys

sys.path.append(os.path.dirname(os.getcwd()))


para_Executor = {
    'Loader': {
        'adult': {
            'filepath': 'benchmark://adult',
            'na_values': {k: '?' for k in [
                'workclass',
                'occupation',
                'native-country'
            ]}
        }
    },
    'Splitter': {
        '0.8': {
            'num_samples': 2,
            'train_split_ratio': 0.8,
        }
    },
    'Processor': {
        'drop-IQR-stanard-label': {
            'missing': {
                'method': 'missing_drop',
                'all': True
            },
            #'method': , # ValueError: y contains previously unseen labels:
            'encoder': [
                {'method': 'encoder_label',
                 'include': ['education','marital-status','relationship','gender']
                },
                {'method': 'encoder_uniform',
                 'include': ['workclass', 'occupation', 'race', 'native-country', 'income']
                }
            ],
            'outlier': {
                'method': 'outlier_iqr',
                'include': 'hours-per-week'
            },
            'scaler': {
                'method': 'scaler_standard',
                'exclude': ['hours-per-week',
                    'workclass', 'education', 'marital-status',
                    'occupation', 'relationship', 'race', 'gender',
                    'native-country', 'income'
                ]
            }
        }
    },
    'Synthesizer': {
        'GaussianCopula': {
            'synthesizing_method': 'sdv-singletable-gaussiancopula'
        }
    },
    'Evaluator': {
        'anonymeter-SinglingOut': {
            'evaluating_method': 'anonymeter-singlingout-univariate',
            'anonymeter_n_attacks': 1,  # 2000'
            'anonymeter_num_samples': 2
        }
    }
}

executor_single = PETsARD.Executor(**para_Executor)
executor_single.run()
pprint(
    executor_single.evaluator[(
        'adult',
        '0.8',
        1,
        'drop-IQR-stanard-label',
        'GaussianCopula',
        'anonymeter-SinglingOut',
        1
    )].Evaluator.evaluation
)

Loader - Benchmarker: file benchmark\adult.csv already exist and match SHA-256.
                      PETsARD will ignore download and use local data directly.
Executor - Loader: adult loading time: 7.0139 sec.
Executor - Splitter: 0.8 splitting time: 0.0689 sec.
No self-defined config passed.  Generate a config automatically.
Executor - Processor (preprocessing): drop-IQR-stanard-label processing time: 0.4141 sec.
Synthesizer (SDV - SingleTable): Metafile loading time: 0.0469 sec.
Synthesizer (SDV - SingleTable): Fitting GaussianCopula.
Synthesizer (SDV - SingleTable): Fitting  GaussianCopula spent 7.8931 sec.
Synthesizer (SDV - SingleTable): Sampling GaussianCopula # 21557 rows (same as raw) in 1.2511 sec.
Executor - Synthesizer: GaussianCopula synthesizing time: 9.1911 sec.
Executor - Processor (postprocessing): drop-IQR-stanard-label processing time: 0.0288 sec.
Evaluator (Anonymeter - SinglingOut - Univariate): Now is SinglingOut - Univariate Evaluator
Evaluator (Anonymeter - Sing

## .run_parallel()
Not applicable now

In [1]:
import os
from pprint import pprint
import sys

sys.path.append('D:\\Dropbox\\89_其他應用\\GitHub\\PETsARD')
os.chdir('D:\\Dropbox\\89_其他應用\\GitHub\\PETsARD\\demo')

import PETsARD


para_Executor = {
    'Loader': {
        'adult': {
            'filepath': 'benchmark://adult',
            'na_values': {k: '?' for k in [
                'workclass',
                'occupation',
                'native-country'
            ]}
        }
    },
    'Splitter': {
        '0.8': {
            'num_samples': 2,
            'train_split_ratio': 0.8,
        }
    },
    'Processor': {
        'drop-IQR-stanard-label': {
            'missingist': {
                'method': 'missingist_drop',
                'all': True
            },
            #'method': , # ValueError: y contains previously unseen labels:
            'encoder': [
                {'method': 'encoder_label',
                 'include': ['education','marital-status','relationship','gender']
                },
                {'method': 'encoder_uniform',
                 'include': ['workclass', 'occupation', 'race', 'native-country', 'income']
                }
            ],
            'outlierist': {
                'method': 'outlierist_iqr',
                'include': 'hours-per-week'
            },
            'scaler': {
                'method': 'scaler_standard',
                'exclude': ['hours-per-week',
                    'workclass', 'education', 'marital-status',
                    'occupation', 'relationship', 'race', 'gender',
                    'native-country', 'income'
                ]
            }
        }
    },
    'Synthesizer': {
        'GaussianCopula': {
            'synthesizing_method': 'sdv-singletable-gaussiancopula'
        }
    },
    'Evaluator': {
        'anonymeter-SinglingOut': {
            'evaluating_method': 'anonymeter-singlingout-univariate',
            'anonymeter_n_attacks': 1,  # 2000'
            'anonymeter_num_samples': 2
        }
    }
}

# Processor contains lambda function, but python couldn't pickle it.
# so Processor .run_parallel() didn't valid after Processor migration.
executor_parallel = PETsARD.Executor(**para_Executor)
executor_parallel.run_parallel()
pprint(
    executor_parallel.evaluator[(
        'adult',
        '0.8',
        1,
        'drop-IQR-stanard-label',
        'GaussianCopula',
        'anonymeter-SinglingOut',
        1
    )].Evaluator.evaluation
)

Loading: 100%|██████████| 1/1 [00:20<00:00, 20.74s/it]


AttributeError: Can't pickle local object 'Processor.__init__.<locals>.<lambda>'

# Un arrangment

In [ ]:
import pandas as pd


def Result(__evaluator):
    import numpy as np
    __dict_result = {}
    __para_to_handle = [('Risk', ['risk()', 'value']), ('Risk_CI_btm', ['risk()', 'ci[0]']), ('Risk_CI_top', ['risk()', 'ci[1]']), ('Attack_Rate', ['results()', 'attack_rate', 'value']), ('Attack_Rate_err', ['results()', 'attack_rate', 'error']), ('Baseline_Rate', ['results()', 'baseline_rate', 'value']), ('Baseline_Rate_err', ['results()', 'baseline_rate', 'error']), ('Control_Rate', ['results()', 'control_rate', 'value']), ('Control_Rate_err', ['results()', 'control_rate', 'error'])
                        ]
    for __key, __attrs in __para_to_handle:
        try:
            __attr_value = __evaluator
            for __attr in __attrs:
                if '()' in __attr:
                    __method_name = __attr.split('(')[0]
                    if hasattr(__attr_value, __method_name):
                        __method = getattr(__attr_value, __method_name)
                        if callable(__method):
                            __attr_value = __method()
                        else:
                            __dict_result[__key] = np.nan
                            break
                    else:
                        __dict_result[__key] = np.nan
                        break
                elif '[' in __attr:
                    __attr_name = __attr.split('[')[0]
                    __index = int(__attr.split('[')[1].rstrip(']'))
                    if hasattr(__attr_value, __attr_name)\
                            and isinstance(getattr(__attr_value, __attr_name), (list, dict, tuple)):
                        try:
                            __attr_value = getattr(
                                __attr_value, __attr_name)[__index]
                        except (IndexError, KeyError):
                            __dict_result[__key] = np.nan
                            break
                    else:
                        __dict_result[__key] = np.nan
                        break
                else:
                    __attr_value = getattr(__attr_value, __attr)
            __dict_result[__key] = __attr_value
        except Exception as e:
            __dict_result[__key] = np.nan
    return __dict_result


for i in ['01', '02', '03']:  # ,'04','05','06'
    from anonymeter.evaluators import SinglingOutEvaluator
    print(f"PETsARD[20231224-085805]_Trial[{i}][Ori].csv")
    evaluator = SinglingOutEvaluator(ori=pd.read_csv(f"PETsARD[20231224-085805]_Trial[{i}][Ori].csv"), syn=pd.read_csv(f"PETsARD[20231224-085805]_Trial[{i}-1-1]Postproc.csv"), control=pd.read_csv(f"PETsARD[20231224-085805]_Trial[{i}][Ctrl].csv"), n_attacks=2000
                                     )
    try:
        evaluator.evaluate(mode='univariate')
        print(Result(evaluator))
    except RuntimeError as ex:
        print(f"Singling out evaluation failed with {ex}."
              "Please re-run this cell."
              "For more stable results increase `n_attacks`. Note that this will "
              "make the evaluation slower.")


In [ ]:
import itertools
for type in ['][Ori]', '][Ctrl]', '-1-1]Postproc']:
    for combo in itertools.combinations(['01', '02', '03', '04', '05', '06'], 2):
        df_a = pd.read_csv(
            f"PETsARD[20231224-085805]_Trial[{combo[0]}{type}.csv")
        df_b = pd.read_csv(
            f"PETsARD[20231224-085805]_Trial[{combo[1]}{type}.csv")
        if df_a.equals(df_b):
            print(type+': '+str(combo))
            print("They're same!!??")
        # else:
            # print(type+': '+str(combo))
            # print("They're different.")
print('done.')


In [ ]:
import pandas as pd


def Result(__evaluator):
    import numpy as np
    __dict_result = {}
    __para_to_handle = [('Risk', ['risk()', 'value']), ('Risk_CI_btm', ['risk()', 'ci[0]']), ('Risk_CI_top', ['risk()', 'ci[1]']), ('Attack_Rate', ['results()', 'attack_rate', 'value']), ('Attack_Rate_err', ['results()', 'attack_rate', 'error']), ('Baseline_Rate', ['results()', 'baseline_rate', 'value']), ('Baseline_Rate_err', ['results()', 'baseline_rate', 'error']), ('Control_Rate', ['results()', 'control_rate', 'value']), ('Control_Rate_err', ['results()', 'control_rate', 'error'])
                        ]
    for __key, __attrs in __para_to_handle:
        try:
            __attr_value = __evaluator
            for __attr in __attrs:
                if '()' in __attr:
                    __method_name = __attr.split('(')[0]
                    if hasattr(__attr_value, __method_name):
                        __method = getattr(__attr_value, __method_name)
                        if callable(__method):
                            __attr_value = __method()
                        else:
                            __dict_result[__key] = np.nan
                            break
                    else:
                        __dict_result[__key] = np.nan
                        break
                elif '[' in __attr:
                    __attr_name = __attr.split('[')[0]
                    __index = int(__attr.split('[')[1].rstrip(']'))
                    if hasattr(__attr_value, __attr_name)\
                            and isinstance(getattr(__attr_value, __attr_name), (list, dict, tuple)):
                        try:
                            __attr_value = getattr(
                                __attr_value, __attr_name)[__index]
                        except (IndexError, KeyError):
                            __dict_result[__key] = np.nan
                            break
                    else:
                        __dict_result[__key] = np.nan
                        break
                else:
                    __attr_value = getattr(__attr_value, __attr)
            __dict_result[__key] = __attr_value
        except Exception as e:
            __dict_result[__key] = np.nan
    return __dict_result


for i in ['01', '02', '03']:  # ,'04','05','06'
    from anonymeter.evaluators import SinglingOutEvaluator
    print(f"PETsARD[20231224-085805]_Trial[{i}][Ori].csv")
    evaluator = SinglingOutEvaluator(ori=pd.read_csv(f"PETsARD[20231224-085805]_Trial[{i}][Ori].csv"), syn=pd.read_csv(f"PETsARD[20231224-085805]_Trial[{i}-1-1]Postproc.csv"), control=pd.read_csv(f"PETsARD[20231224-085805]_Trial[{i}][Ctrl].csv"), n_attacks=2000
                                     )
    try:
        evaluator.evaluate(mode='univariate')
        print(Result(evaluator))
    except RuntimeError as ex:
        print(f"Singling out evaluation failed with {ex}."
              "Please re-run this cell."
              "For more stable results increase `n_attacks`. Note that this will "
              "make the evaluation slower.")


Executor - Loader: adult loading time: 6.8097 sec.
Executor - Splitter: 0.8 splitting time: 0.339 sec.
Preprocessor - Outlierist (IQR): Dropped  1060 rows on fnlwgt         . Kept [-63981.5, 419234.5] only.
Preprocessor - Outlierist (IQR): Dropped   227 rows on educational-num. Kept [3.0, 19.0] only.
Preprocessor - Outlierist (IQR): Dropped  1705 rows on capital-loss   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Dropped  9432 rows on hours-per-week . Kept [32.5, 52.5] only.
Preprocessor - Outlierist (IQR): Dropped   214 rows on age            . Kept [-0.5, 75.5] only.
Preprocessor - Outlierist (IQR): Dropped  3030 rows on capital-gain   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Totally Dropped 13932 in 36207 rows.
Preprocessor - Encoder (Label): Column native-country  been labelized from 0 to 39.
Preprocessor - Encoder (Label): Column gender          been labelized from 0 to  1.
Preprocessor - Encoder (Label): Column race            been labelized from 0 to 

Found 765 failed queries out of 2000. Check DEBUG messages for more details.


Executor - Evaluator: anonymeter-SinglingOut at 1 trials evaluating time: 131.365 sec.
Evaluator (Anonymeter - SinglingOut - Univariate): Now is SinglingOut - Univariate Evaluator
Evaluator (Anonymeter - SinglingOut - Univariate): Evaluator time: 0.0322 sec.
Evaluator (Anonymeter): Evaluating  SinglingOut - Univariate.


Found 802 failed queries out of 2000. Check DEBUG messages for more details.


Executor - Evaluator: anonymeter-SinglingOut at 2 trials evaluating time: 131.1331 sec.
Evaluator (Anonymeter - SinglingOut - Univariate): Now is SinglingOut - Univariate Evaluator
Evaluator (Anonymeter - SinglingOut - Univariate): Evaluator time: 0.0336 sec.
Evaluator (Anonymeter): Evaluating  SinglingOut - Univariate.


...

Found 821 failed queries out of 2000. Check DEBUG messages for more details.


Executor - Evaluator: anonymeter-SinglingOut at 5 trials evaluating time: 132.587 sec.
Evaluator (Anonymeter - SinglingOut - Univariate): Now is SinglingOut - Univariate Evaluator
Evaluator (Anonymeter - SinglingOut - Univariate): Evaluator time: 0.036 sec.
Evaluator (Anonymeter): Evaluating  SinglingOut - Univariate.


Found 800 failed queries out of 2000. Check DEBUG messages for more details.


Executor - Evaluator: anonymeter-SinglingOut at 6 trials evaluating time: 131.8783 sec.
Evaluator (Anonymeter - SinglingOut - Univariate): Now is SinglingOut - Univariate Evaluator
Evaluator (Anonymeter - SinglingOut - Univariate): Evaluator time: 0.0352 sec.
Evaluator (Anonymeter): Evaluating  SinglingOut - Univariate.


Found 799 failed queries out of 2000. Check DEBUG messages for more details.


In [ ]:
import pandas as pd
for i in ['01', '02', '03']:  # ,'04','05','06'
    eval = PETsARD.Evaluator(evaluating_method='anonymeter-singlingout-univariate', data={'ori': pd.read_csv(f"PETsARD[20231224-085805]_Trial[{i}][Ori].csv"), 'syn': pd.read_csv(f"PETsARD[20231224-085805]_Trial[{i}-1-1]Postproc.csv"), 'control': pd.read_csv(f"PETsARD[20231224-085805]_Trial[{i}][Ctrl].csv")
                                                                                          }, anonymeter_n_attacks=500
                             )
    eval.eval()
    print(eval.Evaluator.evaluation)


In [ ]:
import itertools
for type in ['][Ori]', '][Ctrl]', '-1-1]Postproc']:
    for combo in itertools.combinations(['01', '02', '03', '04', '05', '06'], 2):
        df_a = pd.read_csv(
            f"PETsARD[20231224-085805]_Trial[{combo[0]}{type}.csv")
        df_b = pd.read_csv(
            f"PETsARD[20231224-085805]_Trial[{combo[1]}{type}.csv")
        if df_a.equals(df_b):
            print(type+': '+str(combo))
            print("They're same!!??")
        # else:
            # print(type+': '+str(combo))
            # print("They're different.")
print('done.')


Executor - Evaluator: anonymeter-SinglingOut at 7 trials evaluating time: 131.5421 sec.
Evaluator (Anonymeter - SinglingOut - Univariate): Now is SinglingOut - Univariate Evaluator
Evaluator (Anonymeter - SinglingOut - Univariate): Evaluator time: 0.0354 sec.
Evaluator (Anonymeter): Evaluating  SinglingOut - Univariate.


In [ ]:
{
    'Loader': {
        'NHANES': {
            'filepath': '../[sunset]/data/[NHANES] B.csv',
            'header_exist': False,
            'header_names': ['gen', 'age', 'race', 'edu', 'mar', 'bmi', 'dep', 'pir', 'gh', 'mets', 'qm', 'dia']
        }
    }
}
